#Client E-commerce

Here below you can find a script Python to simulate a typical environment. Data is sended in batches to our API for traning the model while our prediction API is triggered anytime someone log-in.

In [2]:
import pandas as pd
import regex as re
import requests
import time
import json 

finalDB = pd.read_csv('https://raw.githubusercontent.com/omkarbhoite25/Juction-Hackathon-Columbia-Road/main/datasets/transaction_best_costumers.csv')
API_ENDPOINT = "https://predicto.pythonanywhere.com/update/transaction"
API_ENDPOINT_2 = "https://predicto.pythonanywhere.com/get/suggestion"
total = 0
suc = 0
timestamp = sorted(set(finalDB['TIMESTAMP']))
#timestamp = filter(lambda x: x < 1163452400, timestamp)

for t in range(len(timestamp)-1):
  batch = finalDB[finalDB['TIMESTAMP']==timestamp[t]]
  batch = batch[['PROD_CODE','PROD_CODE_10','CUST_CODE','CUST_LIFESTAGE','BASKET_ID']]
  batch_json = []
  batch = batch.groupby(['CUST_CODE','CUST_LIFESTAGE','BASKET_ID']).sum()
  order_obj = {}
  for index, row in batch.iterrows():
    order_obj['id'] = index[2]
    order_obj['timestamp'] = timestamp[t] 
    order_obj['customer'] = index[0] 
    order_obj['customer_info'] = index[1]
    order_obj['items'] = []
    prods = re.findall("PRD\d{6}", row['PROD_CODE'])
    categs = re.findall("C\w\d{5}", row['PROD_CODE_10'])
    for i in range(len(prods)):
      basket_obj = {
          'prod_id': prods[i],
          'cat_id': categs[i]
      }
      order_obj['items'].append(basket_obj)
      print(order_obj)
    batch_json.append(order_obj)
  r = requests.post(url = API_ENDPOINT, json=batch_json)
  if r.status_code != 200:
    print("FAILED")
  print(time.strftime('UPDATE %Y-%m-%d', time.localtime(timestamp[t])))

  ## ASK FOR PREDICTION
  batch = finalDB[finalDB['TIMESTAMP']==timestamp[t+1]]
  batch = batch[['PROD_CODE_10','CUST_CODE']]
  today_customers = set(batch['CUST_CODE'])
  for customer in today_customers:
    PARAMS = {'customer': customer, 'timestamp': timestamp[t+1]}
    r = requests.get(url = API_ENDPOINT_2, params = PARAMS)
    resp = json.loads(r.text)
    rec_prod = resp['Products']
    rec_cat = resp['Categories']
    if len(rec_prod) > 0:
      total += 1
    for p in rec_cat:
      future_purch = batch[batch['CUST_CODE']==customer]['PROD_CODE_10'].to_list()
      if p in future_purch:
        print("*************\nSUCCESS!! " + customer + " REALLY WANTED " + rec_prod[0] + "\n****************")
        suc += 1
      else:
        print("PREDICTION FAILED")
    time.sleep(0.3)

print()
print("ACCURACY: " + str(suc/total))

{'id': 994100100570459, 'timestamp': 1144627200, 'customer': 'CUST0000696999', 'customer_info': 'YF', 'items': [{'prod_id': 'PRD090361', 'cat_id': 'CL00188'}]}
UPDATE 2006-04-10
{'id': 994100100488827, 'timestamp': 1144800000, 'customer': 'CUST0000561725', 'customer_info': 'OT', 'items': [{'prod_id': 'PRD090126', 'cat_id': 'CL00030'}]}
UPDATE 2006-04-12
{'id': 994100100389361, 'timestamp': 1145059200, 'customer': 'CUST0000395612', 'customer_info': 'OA', 'items': [{'prod_id': 'PRD090199', 'cat_id': 'CL00079'}]}
{'id': 994100100389361, 'timestamp': 1145059200, 'customer': 'CUST0000395612', 'customer_info': 'OA', 'items': [{'prod_id': 'PRD090199', 'cat_id': 'CL00079'}, {'prod_id': 'PRD090435', 'cat_id': 'CL00079'}]}
{'id': 994100100399363, 'timestamp': 1145059200, 'customer': 'CUST0000412519', 'customer_info': 'OA', 'items': [{'prod_id': 'PRD090305', 'cat_id': 'CL00031'}]}
{'id': 994100100399363, 'timestamp': 1145059200, 'customer': 'CUST0000412519', 'customer_info': 'OA', 'items': [{'pro

KeyboardInterrupt: ignored

As can be seen by the output, the accuracy is reaching 75% with a very small dataset giving us the belief our bayesian model can be the right path to follow for many improvement. 